## DE Final Project Transformation

In [3]:
import pandas as pd

In [ ]:
cleaned_data = pd.read_csv('/Users/kevinko/Desktop/Uchicago/Data engineering platform/Final Project/Crimes_-_2001_to_Present_20231106.csv')

In [ ]:
cleaned_data = cleaned_data.dropna()

In [ ]:
cleaned_data.head()

## Cleaning the unnecessary columns 

In [ ]:
cleaned_data.columns

In [ ]:
columns_to_drop = ['Case Number', 'Block', 'IUCR',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location']

In [ ]:
cleaned_data = cleaned_data.drop(columns=columns_to_drop)

In [ ]:
cleaned_data 

## Convert the date to quarter

In [ ]:
import pandas as pd

# Convert 'Date' column to datetime format
cleaned_data['Date'] = pd.to_datetime(cleaned_data['Date'])

# Create 'year' column
cleaned_data['year'] = cleaned_data['Date'].dt.year

# Create 'quarter' column
cleaned_data['quarter'] = cleaned_data['Date'].dt.quarter

print(cleaned_data[cleaned_data['quarter']==1])

In [ ]:
cleaned_data

## Convernt community area into neighborhood
## !! Not a necessary step for normalizing data but will be useful for data analysis

In [ ]:
community = pd.read_csv('/Users/kevinko/Desktop/Uchicago/Data engineering platform/Final Project/chicago_community_areas_transposed.csv')

In [ ]:
community

In [ ]:
# Assuming 'cleaned_data' DataFrame has a column 'Community Area'
# Assuming 'community' DataFrame has a column 'neighborhood_id'

# Perform a merge based on the common columns
merged_data = cleaned_data.merge(community, left_on='Community Area', right_on='community_area', how='left')

# 'how' parameter determines the type of merge. Here 'left' ensures all rows from 'cleaned_data' are kept

# Display the merged DataFrame
print(merged_data)

cleaned_data = merged_data # make the data back again 


## Convert Crime type to valid score

In [ ]:
crime = pd.read_excel('/Users/kevinko/Desktop/Uchicago/Data engineering platform/Final Project/Criminal type numerical scale_cleaned.xlsx')

In [ ]:
crime.columns

In [ ]:
merged_data_2 = cleaned_data.merge(crime, left_on='Primary Type', right_on='Primary Type', how='left')


# Display the merged DataFrame
print(merged_data_2)


In [ ]:
merged_data_2.columns

In [ ]:
drop_column_2 = ['District', 'Ward', 'Date']

In [ ]:
cleaned_crime_data = merged_data_2.drop(columns=drop_column_2)

In [ ]:
cleaned_crime_data


In [ ]:
average_scores = cleaned_crime_data.groupby(['quarter', 'year', 'Community Area'])['Score'].mean().reset_index()

print(average_scores.round(2))


In [ ]:
df = pd.merge(cleaned_crime_data, average_scores.round(2), on=['quarter', 'year', 'Community Area'], how='left', suffixes=('', '_avg'))

## renaming name to neighbourhood

In [ ]:
df = df.rename(columns={'Community Area': 'neighbourhood_id'})

## Final drop (normalization of Crime Data)

In [ ]:
drop_column_final = [ 'Primary Type', 'year']

df = df.drop(columns=drop_column_final)

In [ ]:
df

In [ ]:
df.to_csv('cleaned_crime.csv', index=False)

## Airbnb Listing

In [4]:
listing = pd.read_csv('/Users/kevinko/Desktop/Uchicago/Data engineering platform/Final Project/listing.csv')

In [6]:
listing = listing.dropna(subset=['neighbourhood_id'])

listing['neighbourhood_id'] = listing['neighbourhood_id'].astype(int)

 need to make sure that the combination of 'neighbourhood_id' and 'quarter' is unique in both tables. If there are multiple rows with the same combination in either table, a Cartesian product will be created, resulting in additional rows.

In [8]:
df = df.drop_duplicates(subset=['neighbourhood_id', 'quarter'], keep='first')


In [10]:
df

,ID,neighbourhood_id,quarter,Score,Score_avg
0,12876758,3,4,7,5.82
1,12876833,1,4,5,5.86
2,12877365,71,4,8,6.12
3,12877003,38,4,5,5.90
4,12877755,33,4,5,5.88
...,...,...,...,...,...
165042,13126416,34,3,7,6.18
165171,13135398,50,3,5,5.82
165325,13127006,74,3,6,5.65
165500,13127292,11,3,8,5.69


In [9]:
listing

,listing_id,host_id,roomtype_id,neighbourhood_id,license_id,avg_price,min_nights,review_per_month,occupancy,quarter
0,2384,2613,1,41,1,90,3,2.13,18,1
1,94450,504470,2,77,1,65,7,0.88,187,1
2,145659,683529,2,5,1,198,2,0.65,35,1
3,7126,17928,2,24,1,85,2,2.89,86,1
4,189821,899757,2,22,1,202,3,4.27,158,1
...,...,...,...,...,...,...,...,...,...,...
31909,771880753874742770,488113016,2,2,1,66,1,0.00,295,4
31910,771881329605160225,488113016,2,1,1,92,1,0.00,299,4
31911,779992322133479260,490950442,2,8,2,33,60,0.00,256,4
31912,780097517605219739,316658141,1,32,1,143,4,0.00,307,4


In [11]:
final = pd.merge(listing, df[['neighbourhood_id', 'quarter', 'Score_avg']], 
                     on=['neighbourhood_id', 'quarter'], how='left')

In [ ]:
final.columns

In [ ]:
final.head()

In [12]:
final

,listing_id,host_id,roomtype_id,neighbourhood_id,license_id,avg_price,min_nights,review_per_month,occupancy,quarter,Score_avg
0,2384,2613,1,41,1,90,3,2.13,18,1,5.62
1,94450,504470,2,77,1,65,7,0.88,187,1,5.57
2,145659,683529,2,5,1,198,2,0.65,35,1,5.71
3,7126,17928,2,24,1,85,2,2.89,86,1,5.74
4,189821,899757,2,22,1,202,3,4.27,158,1,5.63
...,...,...,...,...,...,...,...,...,...,...,...
31660,771880753874742770,488113016,2,2,1,66,1,0.00,295,4,5.90
31661,771881329605160225,488113016,2,1,1,92,1,0.00,299,4,5.86
31662,779992322133479260,490950442,2,8,2,33,60,0.00,256,4,5.62
31663,780097517605219739,316658141,1,32,1,143,4,0.00,307,4,5.60


In [21]:
try:
    df.to_csv('crime_cleaned.csv', index=False)
    print("CSV file created successfully.")
except Exception as e:
    print("Error:", e)


CSV file created successfully.


In [ ]:
final.to_csv('listing_cleaned.csv', index=False)

## Normalization to fit our EER

In [ ]:
columns_to_drop = ['ID']
final = final.drop(columns=columns_to_drop)

In [ ]:
final